In [1]:
from typing import TypedDict

# GraphState 상태를 저장하는 용도로 사용합니다.
class GraphState(TypedDict):
    filepath: str  # path
    filetype: str  # pdf
    page_numbers: list[int]  # page numbers
    batch_size: int  # batch size
    split_filepaths: list[str]  # split files
    analyzed_files: list[str]  # analyzed files
    page_elements: dict[int, dict[str, list[dict]]]  # page elements
    page_metadata: dict[int, dict]  # page metadata
    page_summary: dict[int, str]  # page summary
    images: list[str]  # image paths
    images_summary: list[str]  # image summary
    tables: list[str]  # table
    tables_summary: dict[int, str]  # table summary
    texts: list[str]  # text
    texts_summary: list[str]  # text summary

In [2]:
import os
import pymupdf
import json
import requests
from PIL import Image

def split_pdf(state: GraphState):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할합니다.

    :param state: GraphState 객체, PDF 파일 경로와 배치 크기 정보를 포함
    :return: 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체
    """
    # PDF 파일 경로와 배치 크기 추출
    filepath = state["filepath"]
    batch_size = state["batch_size"]

    # PDF 파일 열기
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    # PDF 분할 작업 시작
    for start_page in range(0, num_pages, batch_size):
        # 배치의 마지막 페이지 계산 (전체 페이지 수를 초과하지 않도록)
        end_page = min(start_page + batch_size, num_pages) - 1

        # 분할된 PDF 파일명 생성
        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        # 새로운 PDF 파일 생성 및 페이지 삽입
        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 원본 PDF 파일 닫기
    input_pdf.close()

    # 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체 반환
    return GraphState(split_filepaths=ret)

In [4]:
state = GraphState(filepath="C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting.pdf", batch_size=10)
state_out = split_pdf(state)
state.update(state_out)
state

총 페이지 수: 8
분할 PDF 생성: C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting_0000_0007.pdf


{'filepath': 'C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting.pdf',
 'batch_size': 10,
 'split_filepaths': ['C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/A review of pediatric fasting guidelines and strategies to help children manage preoperative fasting_0000_0007.pdf']}

In [3]:
from unstructured.partition.pdf import partition_pdf
import nltk

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

file_path = "C:/Users/user/Desktop/BOAZ_ADV/Wang_Gyu/635.full.pdf"

elements = partition_pdf(file_path,
                          strategy="hi_res",
                          max_partition=1)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?